Tips: 

https://scikit-learn.org/stable/modules/feature_extraction.html


1 Word removal-experiment with word removals both in high frequency and low frequency space

2 Taxonomy- Ordered heirarchial taxonomy can help you reduce entropy in content. Similar to what decision trees accomplish. This is SUPER unappreciated in most text analytics literature

3 Lemmatization- it sometimes helps improve structure of text

4 Sampling - Are your classes balanced? if not resample, create synthetic data to populate low frequency classes

5 ensemble/boosting/bagging methods- Have you tried meta algos to improve your performance?

6 Data Quality check- Good data is like good food. Have you checked if you have any useless data? pruning data set helps improve structure a lot!

7 Hyper Parameter tuning- Did you exhaustively search the subspaces to fine tune the model. Most models show upto 10% improvement by just hyperparameter tuning.

8 Word Gram conversion- you can improve feature set by converting important bigrams and trigrams into unigrams "food processing" can be changed to "food_processing"

9 Data preparation- Count vectors, tf-idf, word2vec .. et al.. Naive bayes seems to work better Count vectors,

10 Sample length- This is a quirk i noticed in text mining. if you are classifying sentences ranging from few words to lets say a paragraph. With more additional words algos like naive bayes dont work well due lack of convergence arising from positive hits they get from all words for more that one class to put it naively (pun intended :P)

11 Last point I can think is instead of getting class assignment get class probabilities and use that itself a feature in your ensembles. It seems to work in a variety of cases.




<h1>Pro/against Brexit comments classfier - DAT340 / DIT866</h1>

* You should include instructions for running your classifier on a separate test set


    How much consensus is there between annotators of the dataset?
    -> waiting for 2nd round of annotation
    How do you represent your data as features?
    Did you process the features in any way?
    Did you bring in any additional sources of data?
    How did you select which learning algorithms to use?
    Did you try to tune the hyperparameters of the learning algorithm, and in that case how?
    How do you evaluate the quality of your system?
    How well does your system compare to a stupid baseline?
    Can you say anything about the errors that the system makes? For a classification task, you may consider a confusion matrix.
    Is it possible to say something about which features the model considers important? (Whether this is possible depends on the type of classifier you are using.) 

In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.svm import NuSVC
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier

<h3> Read the data from the file </h3>
Splits data into training and test set (80% - training set, 20% - testing set) 

In [2]:
def read_data(filename):
    X = []
    Y = []
    with open(filename, encoding = 'utf-8') as f: 
        for l in f:
            cols = l.split('\t')
            Y.append(cols[0]) #outputs
            X.append(cols[1]) #features   
    return X, Y

trainDataSet = "datasets/a2_train.tsv" 
smallDataSet = "datasets/a2_first_sample.tsv"
testSetFraction = 0.2

X,Y = read_data(trainDataSet)
#X,Y = read_data(smallDataSet)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)


print("Elements in training set: " + str(len(Xtrain)))
print("Elements in test set: " + str(len(Xtest)))

Elements in training set: 5787
Elements in test set: 1447


### Lemmatization - preprocessing

In [3]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import re

porter = PorterStemmer()
lancester = LancasterStemmer()

#[porter.stem(word) for word in text.split()]



In [4]:
#class TfidfVectorizerCustom(TfidfVectorizer):



#print(TfidfVectorizerCustom.get_feature_names())

### Helpers

In [5]:
# prints best classifiers         
def bestClassifier(scores):
    print(type(scores))
   
     
    for s in scores:
        print(s[0] + "\t" + str((s[1][0])*100)[:6]+" "+str((s[1][1])*100)[:6] +" "+ str((s[1][2])*100)[:6])
        print(len(scores))



### Find the best classfier (sparse matrix)

In [6]:
names = ["ExtraTreesClassifier:",
         "GradientBoostingClas:",
         "LogisticRegressionCV:",
         "LogisticRegression:",
         "SVC:              ",
         "SGDClassifier:    ",
         "NuSVC:            ",
         "DecisionTreeClass:",
         "ExtraTreeClassifier:",
         "CalibratedClasCV:",
         "Nearest Neighbors:",
         "BaggingClassifier:",
         "Linear SVM:      ",
         "RBF SVM:          ",
         "Decision Tree:    ",
         "Random Forest:    ",
         "Perceptron:       ",
         "AdaBoost:        ",
         #"Neural Network:   "
         ]

classifiers = [
        ExtraTreesClassifier(),
        GradientBoostingClassifier(),
        LogisticRegressionCV(),
        LogisticRegression(),
        SVC(),
        SGDClassifier(),
        NuSVC(),
        DecisionTreeClassifier(),
        ExtraTreeClassifier(),
        CalibratedClassifierCV(),
        KNeighborsClassifier(3),
        BaggingClassifier(),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        Perceptron(),
        AdaBoostClassifier(),
        #MLPClassifier(alpha=1),
        ]

result = zip(names, classifiers)
scores = []

for n , cls in result:
    pipeline = Pipeline([
        ('vect', TfidfVectorizer()),
        ('clf', cls)])
    
    results = cross_validate(pipeline, Xtrain, Ytrain)
    
    #evaluating
    pipeline.fit(Xtrain, Ytrain)
    Yguess = pipeline.predict(Xtest)
    scores.append((n, (accuracy_score(Ytest, Yguess))))

    #sorting results
sortedResults = sorted(scores, key=lambda tup: tup[1], reverse=True)
    
for item in sortedResults:
    print(item[0] + '\t' + str((item[1]) * 100 )[:5]+ '%')

SGDClassifier:    	76.36%
RBF SVM:          	76.08%
LogisticRegression:	76.01%
LogisticRegressionCV:	75.88%
CalibratedClasCV:	75.60%
ExtraTreesClassifier:	72.49%
Perceptron:       	72.42%
NuSVC:            	71.94%
GradientBoostingClas:	71.45%
AdaBoost:        	70.14%
BaggingClassifier:	69.52%
Nearest Neighbors:	65.79%
DecisionTreeClass:	64.82%
Decision Tree:    	64.68%
Linear SVM:      	63.23%
ExtraTreeClassifier:	63.09%
SVC:              	52.03%
Random Forest:    	51.96%
